In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from IPython.display import clear_output
import pandas as pd

In [2]:
# set up selenium webscraping browser

option = webdriver.ChromeOptions()
option.add_argument('incognito')

browser = webdriver.Chrome(options=option)

In [3]:
# scrape esrb.org for game information

pg = 1
games_list = []

while True:
    browser.get("https://www.esrb.org/search/?searchKeyword=" \
                "&platform=Nintendo%20Switch%2CPlayStation%204%2CXbox%20One" \
                "&rating=E%2CE10%2B%2CT%2CM%2CAO&descriptor=All%20Content" \
                "&pg={}&searchType=All".format(pg))
    try:
        # make selenium wait until games are loaded before moving on
        element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'game')))
        
        # games on current page
        results = browser.find_elements_by_xpath("//div[@class='game']")
        
        titles, consoles, ratings, descriptors = [], [], [], []
        
        # pull relevant content from results
        for x in results:
            # titles
            title = x.find_element_by_css_selector('h2').text
            titles.append(title)
            
            # consoles
            console = x.find_element_by_class_name('platforms').text
            consoles.append(console)
            
            # ratings are displayed on the page as an image,
            # so we have to pull the rating from the image's url
            xpath = x.find_element_by_css_selector('img')
            rating = xpath.get_attribute('src')[58:-4]
            ratings.append(rating)
            
            # descriptors
            desc = x.find_elements_by_css_selector('td')[1].text
            descriptors.append(desc)
            
        # collect the results as a list of dictionaries
        for i in range(len(titles)):
            game_dict = {}
            game_dict['title'] = titles[i]
            game_dict['consoles'] = consoles[i]
            game_dict['rating'] = ratings[i]
            game_dict['descriptors'] = descriptors[i]
            games_list.append(game_dict)
            
        # progress indicator
        clear_output(wait=True)
        print('Page: {}'.format(pg))
        pg += 1
        
    # stop running when no more games are found
    except:
        print('Stopped on page {}'.format(pg))
        break

Page: 521
Stopped on page 522


In [4]:
# preview of games_list
games_list[:5]

[{'title': 'The Sisters - Party of the Year',
  'consoles': 'PlayStation 4, PlayStation 5, Nintendo Switch',
  'rating': 'E',
  'descriptors': 'Mild Cartoon Violence'},
 {'title': 'Chicken Police - Paint it RED!',
  'consoles': 'Windows PC, PlayStation 4, Nintendo Switch',
  'rating': 'M',
  'descriptors': 'Blood, Drug Reference, Partial Nudity, Sexual Themes, Strong Language, Violence'},
 {'title': 'Vader Immortal',
  'consoles': 'PlayStation 4',
  'rating': 'T',
  'descriptors': 'Animated Blood, Violence'},
 {'title': 'Psychonauts 2',
  'consoles': 'Linux, Macintosh, Windows PC, PlayStation 4, Xbox One, Xbox Series',
  'rating': 'T',
  'descriptors': 'Alcohol Reference, Crude Humor, Fantasy Violence, Mild Language, Mild Suggestive Themes, Use of Tobacco'},
 {'title': 'Diablo II: Resurrected',
  'consoles': 'Windows PC, PlayStation 4, PlayStation 5, Nintendo Switch, Xbox One, Xbox Series',
  'rating': 'M',
  'descriptors': 'Blood and Gore, Suggestive Themes, Violence'}]

In [5]:
# dataframe of collected data
df = pd.DataFrame(games_list)
df.head()

,title,consoles,rating,descriptors
0,The Sisters - Party of the Year,"PlayStation 4, PlayStation 5, Nintendo Switch",E,Mild Cartoon Violence
1,Chicken Police - Paint it RED!,"Windows PC, PlayStation 4, Nintendo Switch",M,"Blood, Drug Reference, Partial Nudity, Sexual ..."
2,Vader Immortal,PlayStation 4,T,"Animated Blood, Violence"
3,Psychonauts 2,"Linux, Macintosh, Windows PC, PlayStation 4, X...",T,"Alcohol Reference, Crude Humor, Fantasy Violen..."
4,Diablo II: Resurrected,"Windows PC, PlayStation 4, PlayStation 5, Nint...",M,"Blood and Gore, Suggestive Themes, Violence"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5204 entries, 0 to 5203
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        5204 non-null   object
 1   consoles     5204 non-null   object
 2   rating       5204 non-null   object
 3   descriptors  5204 non-null   object
dtypes: object(4)
memory usage: 162.8+ KB


In [7]:
# split consoles and descriptors columns
df.descriptors = df.descriptors.map(lambda x: x.split(', '))
df.consoles = df.consoles.map(lambda x: x.split(', '))

df.head()

,title,consoles,rating,descriptors
0,The Sisters - Party of the Year,"[PlayStation 4, PlayStation 5, Nintendo Switch]",E,[Mild Cartoon Violence]
1,Chicken Police - Paint it RED!,"[Windows PC, PlayStation 4, Nintendo Switch]",M,"[Blood, Drug Reference, Partial Nudity, Sexual..."
2,Vader Immortal,[PlayStation 4],T,"[Animated Blood, Violence]"
3,Psychonauts 2,"[Linux, Macintosh, Windows PC, PlayStation 4, ...",T,"[Alcohol Reference, Crude Humor, Fantasy Viole..."
4,Diablo II: Resurrected,"[Windows PC, PlayStation 4, PlayStation 5, Nin...",M,"[Blood and Gore, Suggestive Themes, Violence]"


In [8]:
# choose consoles
consoles_list = ['PlayStation 4', 'Xbox One', 'Nintendo Switch']
for i in range(len(df)):
    df.consoles[i] = [x for x in df.consoles[i] if x in consoles_list]
    
df.head()

,title,consoles,rating,descriptors
0,The Sisters - Party of the Year,"[PlayStation 4, Nintendo Switch]",E,[Mild Cartoon Violence]
1,Chicken Police - Paint it RED!,"[PlayStation 4, Nintendo Switch]",M,"[Blood, Drug Reference, Partial Nudity, Sexual..."
2,Vader Immortal,[PlayStation 4],T,"[Animated Blood, Violence]"
3,Psychonauts 2,"[PlayStation 4, Xbox One]",T,"[Alcohol Reference, Crude Humor, Fantasy Viole..."
4,Diablo II: Resurrected,"[PlayStation 4, Nintendo Switch, Xbox One]",M,"[Blood and Gore, Suggestive Themes, Violence]"


In [9]:
# rearrange dataframe to put target variable at the end
df = df[['title', 'consoles', 'descriptors', 'rating']]
df.head()

,title,consoles,descriptors,rating
0,The Sisters - Party of the Year,"[PlayStation 4, Nintendo Switch]",[Mild Cartoon Violence],E
1,Chicken Police - Paint it RED!,"[PlayStation 4, Nintendo Switch]","[Blood, Drug Reference, Partial Nudity, Sexual...",M
2,Vader Immortal,[PlayStation 4],"[Animated Blood, Violence]",T
3,Psychonauts 2,"[PlayStation 4, Xbox One]","[Alcohol Reference, Crude Humor, Fantasy Viole...",T
4,Diablo II: Resurrected,"[PlayStation 4, Nintendo Switch, Xbox One]","[Blood and Gore, Suggestive Themes, Violence]",M


In [10]:
# pickle it

df.to_pickle('esrb_ratings.pkl')